---
# **LAB 5 - Shared memory (SMEM)**
---

# ▶️ CUDA setup

In [ ]:
!nvcc --version

In [ ]:
!nvidia-smi

## [GPU Compute Capability](https://developer.nvidia.com/cuda-gpus)

## NVCC Plugin for Jupyter notebook

*Usage*:


*   Load Extension `%load_ext nvcc_plugin`
*   Mark a cell to be treated as cuda cell
`%%cuda --name example.cu --compile false`

**NOTE**: The cell must contain either code or comments to be run successfully. It accepts 2 arguments. `-n | --name` - which is the name of either CUDA source or Header. The name parameter must have extension `.cu` or `.h`. Second argument -c | --compile; default value is false. The argument is a flag to specify if the cell will be compiled and run right away or not. It might be usefull if you're playing in the main function

*  We are ready to run CUDA C/C++ code right in your Notebook. For this we need explicitly say to the interpreter, that we want to use the extension by adding `%%cu` at the beginning of each cell with CUDA code. 




In [ ]:
!pip install git+https://github.com/andreinechaev/nvcc4jupyter.git

In [ ]:
%load_ext nvcc_plugin

In [ ]:
# plugin for cpp sintax highlighting 

!wget -O cpp_plugin.py https://gist.github.com/akshaykhadse/7acc91dd41f52944c6150754e5530c4b/raw/cpp_plugin.py
%load_ext cpp_plugin

Clone GPUcomputing site on github...

In [ ]:
!git clone https://github.com/giulianogrossi/GPUcomputing.git

# ▶️ VS Code on Colab

In [ ]:
#@title Colab-ssh tunnel
#@markdown Execute this cell to open the ssh tunnel. Check [colab-ssh documentation](https://github.com/WassimBenzarti/colab-ssh) for more details.

# Install colab_ssh on google colab
!pip install colab_ssh --upgrade --quiet

from colab_ssh import launch_ssh_cloudflared, init_git_cloudflared
ssh_tunnel_password = "gpu" #@param {type: "string"}
launch_ssh_cloudflared(password=ssh_tunnel_password)

# Optional: if you want to clone a Github or Gitlab repository
repository_url="https://github.com/giulianogrossi/GPUcomputing" #@param {type: "string"}
init_git_cloudflared(repository_url)

Define some paths...

In [ ]:
# path setup
!mkdir -p /content/GPUcomputing/lab2
%cd /content/GPUcomputing/lab2
!mkdir -p src


# ▶️ DeviceQuery

In [ ]:
# DeviceQuery dell'attuale device (su Colab!)
!nvcc GPUcomputing/utils/deviceQuery.cu -o deviceQuery
!./deviceQuery

# ✅ Parallel reduction con SMEM


In [ ]:
%%cuda --name preduceSMEM.cu

#include "../GPUcomputing/utils/common.h"
#include <cuda_runtime.h>
#include <stdio.h>
#define DIM 1024

/*
 * An example of using shared memory to optimize performance of a parallel
 * reduction by constructing partial results for a thread block in shared memory
 * before flushing to global memory.
 */

extern __shared__ int dsmem[];

//# Recursive Implementation of Interleaved Pair Approach
int recursiveReduce(int *data, int const size) {
  if (size == 1) return data[0];

  int const stride = size / 2;

  for (int i = 0; i < stride; i++)
    data[i] += data[i + stride];

  return recursiveReduce(data, stride);
}

__device__ void warpReduce(volatile int *smem, unsigned int tid) {
  smem[tid] = min(smem[tid],smem[tid+32]);
  smem[tid] = min(smem[tid],smem[tid+16]);
  smem[tid] = min(smem[tid],smem[tid+8]);
  smem[tid] = min(smem[tid],smem[tid+4]);
  smem[tid] = min(smem[tid],smem[tid+2]);
  smem[tid] = min(smem[tid],smem[tid+1]);
}

//# unroll4 + complete unroll for loop + gmem
__global__ void reduceGmem(int *g_idata, int *g_odata, unsigned int n) {
    //# set thread ID
    unsigned int tid = threadIdx.x;
    int *idata = g_idata + blockIdx.x * blockDim.x;

    //# boundary check
    unsigned int idx = blockIdx.x * blockDim.x + threadIdx.x;

    if (idx >= n) return;

    //# in-place reduction in global memory
    if (blockDim.x >= 1024 && tid < 512) idata[tid] += idata[tid + 512];

    __syncthreads();

    if (blockDim.x >= 512 && tid < 256) idata[tid] += idata[tid + 256];

    __syncthreads();

    if (blockDim.x >= 256 && tid < 128) idata[tid] += idata[tid + 128];

    __syncthreads();

    if (blockDim.x >= 128 && tid < 64) idata[tid] += idata[tid + 64];

    __syncthreads();

    // unrolling warp
    if (tid < 32) 
      warpReduce(idata, tid);

    // write result for this block to global mem
    if (tid == 0) g_odata[blockIdx.x] = idata[0];
}

__global__ void reduceSmem(int *g_idata, int *g_odata, unsigned int n) {
    __shared__ int smem[DIM];

    // set thread ID
    unsigned int tid = threadIdx.x;

    // boundary check
    unsigned int idx = blockIdx.x * blockDim.x + threadIdx.x;

    if (idx >= n) return;

    // convert global data pointer to the local pointer of this block
    int *idata = g_idata + blockIdx.x * blockDim.x;

    // set to smem by each threads
    smem[tid] = idata[tid];
    __syncthreads();

    // in-place reduction in shared memory
    if (blockDim.x >= 1024 && tid < 512) smem[tid] += smem[tid + 512];

    __syncthreads();

    if (blockDim.x >= 512 && tid < 256) smem[tid] += smem[tid + 256];

    __syncthreads();

    if (blockDim.x >= 256 && tid < 128) smem[tid] += smem[tid + 128];

    __syncthreads();

    if (blockDim.x >= 128 && tid < 64)  smem[tid] += smem[tid + 64];

    __syncthreads();

    // unrolling warp
    if (tid < 32)
      warpReduce(smem, tid);

    // write result for this block to global mem
    if (tid == 0) g_odata[blockIdx.x] = smem[0];
}

__global__ void reduceSmemDyn(int *g_idata, int *g_odata, unsigned int n) {
    extern __shared__ int smem[];

    // set thread ID
    unsigned int tid = threadIdx.x;
    int *idata = g_idata + blockIdx.x * blockDim.x;

    // set to smem by each threads
    smem[tid] = idata[tid];
    __syncthreads();

    // in-place reduction in global memory
    if (blockDim.x >= 1024 && tid < 512)  smem[tid] += smem[tid + 512];

    __syncthreads();

    if (blockDim.x >= 512 && tid < 256)  smem[tid] += smem[tid + 256];

    __syncthreads();

    if (blockDim.x >= 256 && tid < 128) smem[tid] += smem[tid + 128];

    __syncthreads();

    if (blockDim.x >= 128 && tid < 64) smem[tid] += smem[tid + 64];

    __syncthreads();

    // unrolling warp
    if (tid < 32) 
      warpReduce(smem, tid);

    // write result for this block to global mem
    if (tid == 0) g_odata[blockIdx.x] = smem[0];
}

//# unroll4 + complete unroll for loop + gmem
__global__ void reduceGmemUnroll(int *g_idata, int *g_odata, unsigned int n) {
  // set thread ID
  unsigned int tid = threadIdx.x;
  unsigned int idx = blockIdx.x * blockDim.x * 4 + threadIdx.x;

  // convert global data pointer to the local pointer of this block
  int *idata = g_idata + blockIdx.x * blockDim.x * 4;

  // unrolling 4
  if (idx + 3 * blockDim.x < n) {
    int a1 = g_idata[idx];
    int a2 = g_idata[idx + blockDim.x];
    int a3 = g_idata[idx + 2 * blockDim.x];
    int a4 = g_idata[idx + 3 * blockDim.x];
    g_idata[idx] = a1 + a2 + a3 + a4;
  }

  __syncthreads();

  // in-place reduction in global memory
  if (blockDim.x >= 1024 && tid < 512) idata[tid] += idata[tid + 512];

  __syncthreads();

  if (blockDim.x >= 512 && tid < 256) idata[tid] += idata[tid + 256];

  __syncthreads();

  if (blockDim.x >= 256 && tid < 128) idata[tid] += idata[tid + 128];

  __syncthreads();

  if (blockDim.x >= 128 && tid < 64) idata[tid] += idata[tid + 64];

  __syncthreads();

  // unrolling warp
  if (tid < 32)
    warpReduce(idata, tid);

  // write result for this block to global mem
  if (tid == 0) g_odata[blockIdx.x] = idata[0];
}

__global__ void reduceSmemUnroll(int *g_idata, int *g_odata, unsigned int n) {
    // static shared memory
    __shared__ int smem[DIM];

    // set thread ID
    unsigned int tid = threadIdx.x;

    // global index, 4 blocks of input data processed at a time
    unsigned int idx = blockIdx.x * blockDim.x * 4 + threadIdx.x;

    // unrolling 4 blocks
    int tmpSum = 0;

    // boundary check
    if (idx + 4 * blockDim.x <= n) {
      int a1 = g_idata[idx];
      int a2 = g_idata[idx + blockDim.x];
      int a3 = g_idata[idx + 2 * blockDim.x];
      int a4 = g_idata[idx + 3 * blockDim.x];
      tmpSum = a1 + a2 + a3 + a4;
    }

    smem[tid] = tmpSum;
    __syncthreads();

    // in-place reduction in shared memory
    if (blockDim.x >= 1024 && tid < 512) smem[tid] += smem[tid + 512];

    __syncthreads();

    if (blockDim.x >= 512 && tid < 256)  smem[tid] += smem[tid + 256];

    __syncthreads();

    if (blockDim.x >= 256 && tid < 128)  smem[tid] += smem[tid + 128];

    __syncthreads();

    if (blockDim.x >= 128 && tid < 64)   smem[tid] += smem[tid + 64];

    __syncthreads();

    // unrolling warp
    if (tid < 32) 
      warpReduce(smem, tid);

    // write result for this block to global mem
    if (tid == 0) g_odata[blockIdx.x] = smem[0];
}

__global__ void reduceSmemUnrollDyn(int *g_idata, int *g_odata, unsigned int n) {
    extern __shared__ int smem[];

    // set thread ID
    unsigned int tid = threadIdx.x;
    unsigned int idx = blockIdx.x * blockDim.x * 4 + threadIdx.x;

    // unrolling 4
    int tmpSum = 0;

    if (idx + 3 * blockDim.x < n) {
      int a1 = g_idata[idx];
      int a2 = g_idata[idx + blockDim.x];
      int a3 = g_idata[idx + 2 * blockDim.x];
      int a4 = g_idata[idx + 3 * blockDim.x];
      tmpSum = a1 + a2 + a3 + a4;
    }

    smem[tid] = tmpSum;
    __syncthreads();

    // in-place reduction in global memory
    if (blockDim.x >= 1024 && tid < 512)  smem[tid] += smem[tid + 512];

    __syncthreads();

    if (blockDim.x >= 512 && tid < 256)  smem[tid] += smem[tid + 256];

    __syncthreads();

    if (blockDim.x >= 256 && tid < 128) smem[tid] += smem[tid + 128];

    __syncthreads();

    if (blockDim.x >= 128 && tid < 64) smem[tid] += smem[tid + 64];

    __syncthreads();

    // unrolling warp
    if (tid < 32) 
      warpReduce(smem, tid);

    // write result for this block to global mem
    if (tid == 0) g_odata[blockIdx.x] = smem[0];
}

__global__ void reduceNeighboredGmem(int *g_idata, int *g_odata, unsigned int  n) {
    // set thread ID
    unsigned int tid = threadIdx.x;
    unsigned int idx = blockIdx.x * blockDim.x + threadIdx.x;

    // convert global data pointer to the local pointer of this block
    int *idata = g_idata + blockIdx.x * blockDim.x;

    // boundary check
    if (idx >= n) return;

    // in-place reduction in global memory
    for (int stride = 1; stride < blockDim.x; stride *= 2) {
      if ((tid % (2 * stride)) == 0)
        idata[tid] += idata[tid + stride];

      // synchronize within threadblock
      __syncthreads();
    }

    // write result for this block to global mem
    if (tid == 0) g_odata[blockIdx.x] = idata[0];
}

__global__ void reduceNeighboredSmem(int *g_idata, int *g_odata, unsigned int  n) {
    __shared__ int smem[DIM];

    // set thread ID
    unsigned int tid = threadIdx.x;
    unsigned int idx = blockIdx.x * blockDim.x + threadIdx.x;

    // convert global data pointer to the local pointer of this block
    int *idata = g_idata + blockIdx.x * blockDim.x;

    // boundary check
    if (idx >= n) return;

    smem[tid] = idata[tid];
    __syncthreads();

    // in-place reduction in global memory
    for (int stride = 1; stride < blockDim.x; stride *= 2)
    {
        if ((tid % (2 * stride)) == 0)
        {
            smem[tid] += smem[tid + stride];
        }

        // synchronize within threadblock
        __syncthreads();
    }

    // write result for this block to global mem
    if (tid == 0) g_odata[blockIdx.x] = smem[0];
}

int main(int argc, char **argv) {
  // set up device
  int dev = 0;
  cudaDeviceProp deviceProp;
  CHECK(cudaGetDeviceProperties(&deviceProp, dev));
  printf("%s starting reduction at ", argv[0]);
  printf("device %d: %s ", dev, deviceProp.name);
  CHECK(cudaSetDevice(dev));

  bool bResult = false;

  // initialization
  int power = 10;

  // execution configuration
  int blocksize = DIM;   // initial block size

  if (argc >= 2) 
      blocksize = atoi(argv[1]);

  if (argc >= 3)
      power = atoi(argv[2]);

  int size = 1 << power; // total number of elements to reduce
  printf("    with array size %d  ", size);

  dim3 block (blocksize, 1);
  dim3 grid  ((size + block.x - 1) / block.x, 1);
  printf("grid %d block %d\n", grid.x, block.x);

  // allocate host memory
  size_t bytes = size * sizeof(int);
  int *h_idata = (int *) malloc(bytes);
  int *h_odata = (int *) malloc(grid.x * sizeof(int));
  int *tmp     = (int *) malloc(bytes);

  // initialize the array
  for (int i = 0; i < size; i++)
    h_idata[i] = (int)( rand() & 0xFF );

  memcpy (tmp, h_idata, bytes);

  int gpu_sum = 0;

  // allocate device memory
  int *d_idata = NULL;
  int *d_odata = NULL;
  CHECK(cudaMalloc((void **) &d_idata, bytes));
  CHECK(cudaMalloc((void **) &d_odata, grid.x * sizeof(int)));

  // cpu reduction
  int cpu_sum = recursiveReduce (tmp, size);
  printf("cpu reduce          : %d\n", cpu_sum);

  // reduce gmem
  CHECK(cudaMemcpy(d_idata, h_idata, bytes, cudaMemcpyHostToDevice));
  reduceNeighboredGmem<<<grid.x, block>>>(d_idata, d_odata, size);
  CHECK(cudaGetLastError());
  CHECK(cudaMemcpy(h_odata, d_odata, grid.x * sizeof(int), cudaMemcpyDeviceToHost));
  gpu_sum = 0;

  for (int i = 0; i < grid.x; i++) 
    gpu_sum += h_odata[i];

  printf("reduceNeighboredGmem: %d <<<grid %d block %d>>>\n", gpu_sum, grid.x, block.x);

  // reduce gmem
  CHECK(cudaMemcpy(d_idata, h_idata, bytes, cudaMemcpyHostToDevice));
  reduceNeighboredSmem<<<grid.x, block>>>(d_idata, d_odata, size);
  CHECK(cudaGetLastError());
  CHECK(cudaMemcpy(h_odata, d_odata, grid.x * sizeof(int), cudaMemcpyDeviceToHost));
  gpu_sum = 0;

  for (int i = 0; i < grid.x; i++) gpu_sum += h_odata[i];

  printf("reduceNeighboredSmem: %d <<<grid %d block %d>>>\n", gpu_sum, grid.x, block.x);

  // reduce gmem
  CHECK(cudaMemcpy(d_idata, h_idata, bytes, cudaMemcpyHostToDevice));
  reduceGmem<<<grid.x, block>>>(d_idata, d_odata, size);
  CHECK(cudaGetLastError());
  CHECK(cudaMemcpy(h_odata, d_odata, grid.x * sizeof(int), cudaMemcpyDeviceToHost));
  gpu_sum = 0;

  for (int i = 0; i < grid.x; i++) gpu_sum += h_odata[i];

  printf("reduceGmem          : %d <<<grid %d block %d>>>\n", gpu_sum, grid.x, block.x);

  // reduce smem
  CHECK(cudaMemcpy(d_idata, h_idata, bytes, cudaMemcpyHostToDevice));
  reduceSmem<<<grid.x, block>>>(d_idata, d_odata, size);
  CHECK(cudaGetLastError());
  CHECK(cudaMemcpy(h_odata, d_odata, grid.x * sizeof(int), cudaMemcpyDeviceToHost));
  gpu_sum = 0;

  for (int i = 0; i < grid.x; i++) gpu_sum += h_odata[i];

  printf("reduceSmem          : %d <<<grid %d block %d>>>\n", gpu_sum, grid.x, block.x);

  // reduce smem
  CHECK(cudaMemcpy(d_idata, h_idata, bytes, cudaMemcpyHostToDevice));
  reduceSmemDyn<<<grid.x, block, blocksize*sizeof(int)>>>(d_idata, d_odata,
          size);
  CHECK(cudaGetLastError());
  CHECK(cudaMemcpy(h_odata, d_odata, grid.x * sizeof(int), cudaMemcpyDeviceToHost));
  gpu_sum = 0;

  for (int i = 0; i < grid.x; i++) gpu_sum += h_odata[i];

  printf("reduceSmemDyn       : %d <<<grid %d block %d>>>\n", gpu_sum, grid.x, block.x);

  // reduce gmem
  CHECK(cudaMemcpy(d_idata, h_idata, bytes, cudaMemcpyHostToDevice));
  reduceGmemUnroll<<<grid.x / 4, block>>>(d_idata, d_odata, size);
  CHECK(cudaGetLastError());
  CHECK(cudaMemcpy(h_odata, d_odata, grid.x / 4 * sizeof(int), cudaMemcpyDeviceToHost));
  gpu_sum = 0;

  for (int i = 0; i < grid.x / 4; i++) gpu_sum += h_odata[i];

  printf("reduceGmemUnroll4   : %d <<<grid %d block %d>>>\n", gpu_sum, grid.x / 4, block.x);

  // reduce smem
  CHECK(cudaMemcpy(d_idata, h_idata, bytes, cudaMemcpyHostToDevice));
  reduceSmemUnroll<<<grid.x / 4, block>>>(d_idata, d_odata, size);
  CHECK(cudaGetLastError());
  CHECK(cudaMemcpy(h_odata, d_odata, grid.x / 4 * sizeof(int), cudaMemcpyDeviceToHost));
  gpu_sum = 0;

  for (int i = 0; i < grid.x / 4; i++) gpu_sum += h_odata[i];

  printf("reduceSmemUnroll4   : %d <<<grid %d block %d>>>\n", gpu_sum, grid.x / 4, block.x);

  // reduce smem
  CHECK(cudaMemcpy(d_idata, h_idata, bytes, cudaMemcpyHostToDevice));
  reduceSmemUnrollDyn<<<grid.x / 4, block, DIM*sizeof(int)>>>(d_idata, d_odata, size);
  CHECK(cudaGetLastError());
  CHECK(cudaMemcpy(h_odata, d_odata, grid.x / 4 * sizeof(int), cudaMemcpyDeviceToHost));
  gpu_sum = 0;

  for (int i = 0; i < grid.x / 4; i++) gpu_sum += h_odata[i];

  printf("reduceSmemDynUnroll4: %d <<<grid %d block %d>>>\n", gpu_sum, grid.x / 4, block.x);

  // free host memory
  free(h_idata);
  free(h_odata);

  // free device memory
  CHECK(cudaFree(d_idata));
  CHECK(cudaFree(d_odata));

  // reset device
  CHECK(cudaDeviceReset());

  // check the results
  bResult = (gpu_sum == cpu_sum);

  if(!bResult) printf("Test failed!\n");

  return EXIT_SUCCESS;
}


In [ ]:
!nvcc -arch=sm_75  src/preduceSMEM.cu -o preduce
!./preduce 1024 20

In [ ]:
!nvcc -arch=sm_75 --ptxas-options=-v src/preduceSMEM.cu -o preduce
!./preduce 1024 20

# ✅ Moltiplicazione matriciale con SMEM


**Prodotto di matrici con SMEM**

Scrivere un programma CUDA per prodotto matrici $C = A*B$ che usi la SMEM e riduca così il 'traffico' in global mem

**passi:**
1. Definire la SMEM per ogni blocco della matrice $C$
2. Svolgere un ciclo sui blocchi per caricare la SMEM da global mem
3. Sincronizzare -1-
4. Nel ciclo effettuare localmente all’interno di ogni blocco il calcolo del prodotto riga-colonna e caricare su registro
5. sincronizzare -2-
6. Scrivere il risultato finale su matrice prodotto in global mem


# 🔴 TODO

In [ ]:
%%cuda --name matmulSMEM.cu
#include <stdio.h>
#include <stdlib.h>
#include "../GPUcomputing/utils/common.h"

#define IDX(i,j,n) (i*n+j)
#define ABS(x,y) (x-y>=0?x-y:y-x)
#define N 1024
#define P 1024
#define M 1024
#define BLOCK_SIZE 32


/*
 * Kernel for matrix product with static SMEM
 *      C  =  A  *  B
 *    (NxM) (MxP) (PxM)
 */
__global__ void matmulSMEMstatic(float* A, float* B, float* C) {
	
	//# TODO
	
}

/*
 * Kernel for matrix product using dynamic SMEM
 */
__global__ void matmulSMEMdynamic(float* A, float* B, float* C, const uint SMEMsize) {
	
	//# TODO
}

/*
 * Kernel for naive matrix product
 */
__global__ void matmul(float* A, float* B, float* C) {
	// indexes
	int row = blockIdx.y * blockDim.y + threadIdx.y;
	int col = blockIdx.x * blockDim.x + threadIdx.x;

	// each thread computes an entry of the product matrix
	if ((row < N) && (col < M)) {
		float sum = 0;
		for (int k = 0; k < P; k++)
			sum += A[IDX(row, k, P)] * B[IDX(k, col, M)];
		C[IDX(row, col, M)] = sum;
	}
}

/*
 *  matrix product on CPU
 */
void matmulCPU(float* A, float* B, float* C) {
	for (int row = 0; row < N; row++)
		for (int col = 0; col < M; col++) {
			float sum = 0;
			for (int k = 0; k < P; k++)
				sum += A[IDX(row, k, P)] * B[IDX(k, col, M)];
			C[IDX(row, col, M)] = sum;
		}
}

/*
 * Test the device
 */
unsigned long testCUDADevice(void) {
	int dev = 0;

	cudaDeviceSetCacheConfig (cudaFuncCachePreferEqual);
	cudaDeviceProp deviceProp;
	cudaSetDevice(dev);
	cudaGetDeviceProperties(&deviceProp, dev);
	printf("Device %d: \"%s\"\n", dev, deviceProp.name);
	printf("Total amount of shared memory available per block: %lu KB\n",
			deviceProp.sharedMemPerBlock / 1024);
	return deviceProp.sharedMemPerBlock;
}

/*
 * elementwise comparison between two mqdb
 */
void checkResult(float *A, float *B) {
	double epsilon = 1.0E-8;
	bool match = 1;
	for (int i = 0; i < N*M; i++)
		if (ABS(A[i], B[i]) > epsilon) {
			match = 0;
			printf("   * Arrays do not match!\n");
			break;
		}
	if (match)
		printf("   Arrays match\n\n");
}

/*
 * MAIN
 */
int main(void) {
	 // Kernels for matrix product
	 //      C  =  A  *  B
	 //    (NxM) (NxP) (PxM)
	printf("N = %d, M = %d, K = %d\n", N, M, P);
	uint rowA = N, rowB = P;
	uint colA = P, colB = M;
	uint rowC = N, colC = M;
	float *A, *B, *C, *C1;
	float *dev_A, *dev_B, *dev_C;

	// dims
	unsigned long Asize = rowA * colA * sizeof(float);
	unsigned long Bsize = rowB * colB * sizeof(float);
	unsigned long Csize = rowC * colC * sizeof(float);
	
	// malloc host memory
	A = (float*) malloc(Asize);
	B = (float*) malloc(Bsize);
	C = (float*) malloc(Csize);
	C1 = (float*) malloc(Csize);

	// device SMEM available/ test device shared memory
	unsigned long maxSMEMbytes = testCUDADevice();
	

	// malloc device memory
	CHECK(cudaMalloc((void** )&dev_A, Asize));
	CHECK(cudaMalloc((void** )&dev_B, Bsize));
	CHECK(cudaMalloc((void** )&dev_C, Csize));
	printf("Total amount of allocated memory on GPU %.2f MB\n\n", (float)(Asize + Bsize + Csize)/(1024.0*1024.0));

	// fill the matrices A and B
	for (int i = 0; i < N * P; i++) A[i] = 1.0;
	for (int i = 0; i < P * M; i++) B[i] = 1.0;

	/***********************************************************/
	/*                       CPU matmul                       */
	/***********************************************************/
	printf("\n   *** CPU & NAIVE KERNEL ***\n\n");
	double start = seconds();
	matmulCPU(A, B, C);
	printf("   matmul elapsed time CPU = %f\n\n", seconds() - start);


	// copy matrices A and B to the GPU
	CHECK(cudaMemcpy(dev_A, A, Asize, cudaMemcpyHostToDevice));
	CHECK(cudaMemcpy(dev_B, B, Bsize, cudaMemcpyHostToDevice));

	/***********************************************************/
	/*                    GPU naive matmul                     */
	/***********************************************************/
	// grid block dims = shared mem dims = BLOCK_SIZE
	dim3 block(BLOCK_SIZE, BLOCK_SIZE);
	dim3 grid((M + block.x - 1) / block.x, (N + block.y - 1) / block.y);
	start = seconds();
	matmul<<<grid, block>>>(dev_A, dev_B, dev_C);
	CHECK(cudaDeviceSynchronize());
	printf("   Kernel naive matmul elapsed time GPU = %f\n", seconds() - start);

	// copy the array 'C' back from the GPU to the CPU
	CHECK(cudaMemcpy(C1, dev_C, Csize, cudaMemcpyDeviceToHost));
	checkResult(C,C1);

	/***********************************************************/
	/*              GPU matmulSMEM static SMEM               */
	/***********************************************************/
	// grid block dims = shared mem dims = BLOCK_SIZE
	printf("\n   *** USING STATIC SMEM ***\n\n");
	start = seconds();
	matmulSMEMstatic<<<grid, block>>>(dev_A, dev_B, dev_C);
	CHECK(cudaDeviceSynchronize());
	printf("   Kernel matmulSMEM static elapsed time GPU = %f\n", seconds() - start);
	
	// amount of SMEM used
	uint SMEMsize = 2 * BLOCK_SIZE * BLOCK_SIZE * sizeof(float);
		if (maxSMEMbytes < SMEMsize)
			printf("   Shared memory usage WARNING: available: %lu, required: %d bytes\n",	maxSMEMbytes, SMEMsize);
		else
			printf("   Total amount of shared memory required per block %.1f KB\n", (float) SMEMsize / (float) 1024);
	
	// copy the array 'C' back from the GPU to the CPU
	CHECK(cudaMemcpy(C1, dev_C, Csize, cudaMemcpyDeviceToHost));
	checkResult(C,C1);
	
	/***********************************************************/
	/*            GPU matmulSMEMD dynamic SMEM                */
	/***********************************************************/
	// set cache size
	cudaDeviceSetCacheConfig (cudaFuncCachePreferShared);
	printf("\n   *** USING DYNAMIC SMEM ***\n\n");

	// try with various SMEM sizes
	uint sizes[] = {8, 16, 32};
	for (int i = 0; i < 3; i++) {
		uint blockSize = sizes[i];
		block.x = blockSize;
		block.y = blockSize;
		grid.x = (M + block.x - 1) / block.x;
		grid.y = (N + block.y - 1) / block.y;
		uint SMEMsize = blockSize * blockSize;
		uint SMEMbyte = 2 * SMEMsize * sizeof(float);
		start = seconds();
		matmulSMEMdynamic<<< grid, block, SMEMbyte >>>(dev_A, dev_B, dev_C, SMEMsize);
		CHECK(cudaDeviceSynchronize());
		printf("   Kernel matmulSMEM dynamic (SMEM size %d) elapsed time GPU = %f\n", blockSize, seconds() - start);

		// amount of SMEM used
		if (maxSMEMbytes < SMEMbyte)
			printf("   Shared memory usage WARNING: available: %lu, required: %d bytes\n",	maxSMEMbytes, SMEMbyte);
		else
			printf("   Total amount of shared memory required per block %.1f KB\n", (float) SMEMbyte / (float) 1024);

		// copy the array 'C' back from the GPU to the CPU
		CHECK(cudaMemcpy(C1, dev_C, Csize, cudaMemcpyDeviceToHost));
		checkResult(C,C1);
	}

	// free the memory allocated on the GPU
	cudaFree(dev_A);
	cudaFree(dev_B);
	cudaFree(dev_C);

	cudaDeviceReset();
	return EXIT_SUCCESS;
}


In [ ]:
# Compilazione ed esecuzione

!nvcc -arch=sm_70 src/matmulSMEM.cu  -o matmulSMEM
!matmulSMEM

In [ ]:
!ls -la

# ✅ Convoluzione con SMEM

In [ ]:
%%cuda --name conv1D.cu

#include <stdlib.h>
#include <stdio.h>
#include "../GPUcomputing/utils/common.h"

#define MASK_RADIUS  5
#define MASK_SIZE    2 * MASK_RADIUS + 1
#define BLOCK_SIZE   128
#define TILE_WIDTH   BLOCK_SIZE + MASK_SIZE - 1

__device__ __constant__ float d_mask[MASK_SIZE];

void initialData(float*, int);
void movingAverage(float*, int n);
void printData(float*, const int);
void convolutionHost(float*, float*, float*, const int);
void checkResult(float*, float*, int);

/*
 *# kernel for 1D convolution: it holds only if MASK_RADIUS < BLOCK_SIZE
 */
__global__ void convolution1D(float *result, float *data, int n) {
	unsigned int i = blockDim.x * blockIdx.x + threadIdx.x;

	//# shared memory size = BLOCK_SIZE + MASK
	__shared__ float tile[TILE_WIDTH];

	//# boundary
	int left = blockIdx.x * blockDim.x - MASK_RADIUS;
	int right = (blockIdx.x + 1) * blockDim.x;

  //# left halo
	if (threadIdx.x < MASK_RADIUS)                      
		tile[threadIdx.x] = left < 0 ? 0 : data[left + threadIdx.x];

  //# center
	tile[threadIdx.x + MASK_RADIUS] = data[i];

  //# right halo  
	if (threadIdx.x >= blockDim.x - MASK_RADIUS)  
		tile[threadIdx.x + MASK_SIZE - 1] = right >= n ? 0 :
				data[right + threadIdx.x - blockDim.x + MASK_RADIUS];

	__syncthreads();

	//# convolution: tile * mask
	float sum = 0;
	for (int i = -MASK_RADIUS; i <= MASK_RADIUS; i++)
		sum += tile[threadIdx.x + MASK_RADIUS + i] * d_mask[i + MASK_RADIUS];

	//# final result
	result[i] = sum;
}

/*
 * MAIN: convolution 1D host & device
 */
int main(int argc, char **argv) {
	// set up device
	int dev = 0;
	cudaDeviceProp deviceProp;
	CHECK(cudaGetDeviceProperties(&deviceProp, dev));
	printf("starting conv1D at device %d: %s\n", dev, deviceProp.name);
	CHECK(cudaSetDevice(dev));

	// set up array size
	int n = 1 << 24;
	int N = MASK_SIZE;

	printf("Array of size = %.1f MB\n", n/(1024.0*1024.0));

	// mem sizes
	size_t nBytes = n * sizeof(float);
	size_t nBytes_mask = N * sizeof(float);

	// grid configuration
	dim3 block(BLOCK_SIZE);
	dim3 grid((n + BLOCK_SIZE - 1) / BLOCK_SIZE);

	// allocate host memory
	float *h_data = (float *) malloc(nBytes);
	float *h_result = (float *) malloc(nBytes);
	float *result = (float *) malloc(nBytes);
	float *h_mask = (float *) malloc(nBytes_mask);

	//  initialize host array
	movingAverage(h_mask, N);
	initialData(h_data, n);

	// convolution on host
	double start = seconds();
	convolutionHost(h_data, result, h_mask, n);
	double hostElaps = seconds() - start;

	// allocate device memory
	float *d_data, *d_result;
	CHECK(cudaMalloc((void**)&d_data, nBytes));
	CHECK(cudaMalloc((void**)&d_result, nBytes));

	// copy data from host to device
	CHECK(cudaMemcpy(d_data, h_data, nBytes, cudaMemcpyHostToDevice));
	CHECK(cudaMemcpyToSymbol(d_mask, h_mask, nBytes_mask));

	start = seconds();
	convolution1D<<<grid, block>>>(d_result, d_data, n);
	CHECK(cudaDeviceSynchronize());
	double devElaps = seconds() - start;
  printf("Times:\n");
	printf("   - CPU elapsed time = %f\n", hostElaps);
  printf("   - GPU elapsed time = %f\n", devElaps);
  printf("   - Speed-up (ratio) = %f\n", hostElaps / devElaps);

	CHECK(cudaMemcpy(h_result, d_result, nBytes, cudaMemcpyDeviceToHost));

	// check result
	checkResult(h_result, result, n);

	// free host and device memory
	CHECK(cudaFree(d_result));
	CHECK(cudaFree(d_data));
	free(h_data);
	free(h_mask);
	free(h_result);
	free(result);

	// reset device
	CHECK(cudaDeviceReset());
	return EXIT_SUCCESS;
}

void initialData(float *h_data, int n) {
	// initialize the data
	for (int i = 0; i < n; i++)
		h_data[i] = 10.0;
}

void movingAverage(float *h_mask, int n) {
	// initialize mask moving average
	for (int i = 0; i < n; i++)
		h_mask[i] = 1.0 / ((float) n);
	return;
}

void printData(float *a, const int size) {
	printf("\n");
	for (int i = 0; i < size; i++)
		printf("%.2f ", a[i]);
	printf("\n");
	return;
}

void convolutionHost(float *data, float *result, float *mask, const int n) {
	for (int i = 0; i < n; i++) {
		float sum = 0;
		for (int j = 0; j < MASK_SIZE; j++) {
			int idx = i - MASK_RADIUS + j;
			if (idx >= 0 && idx < n)
				sum += data[idx] * mask[j];
		}
		result[i] = sum;
	}
}

void checkResult(float *d_result, float *h_result, int n) {
	double epsilon = 1.0E-8;

	for (int i = 0; i < n; i++)
		if (abs(h_result[i] - d_result[i]) > epsilon) {
			printf("different on entry (%d) |h_result - d_result| >  %f\n", i,
					epsilon);
			break;
		}
}



In [ ]:
# Compilazione ed esecuzione
!nvcc -arch=sm_70  src/conv1D.cu -o conv1D
!./conv1D

# 🔴 TODO

In [ ]:
%%cuda --name conv2D.cu
#include <stdlib.h>
#include <string.h>

#include "../GPUcomputing/utils/common.h"

#define DATA_WIDTH   (20*1024)
#define DATA_HEIGHT  (20*1024)
#define BLOCK_SIZE   8
#define MASK_RADIUS  2
#define MASK_SIZE    (2 * MASK_RADIUS + 1)
#define TILE_WIDTH   (BLOCK_SIZE + MASK_SIZE - 1)
#define DEBUG 0

// constant mem
__constant__ float M_dev[MASK_SIZE*MASK_SIZE];

/*
 * kernel for convolution 2D (it holds only if MASK_RADIUS < BLOCK_SIZE)
 */
__global__ void conv2D(float *A, float *B) {
	
	//# TODO
}

/*
 * Average filter
 */
void Avg_mask(float *mask) {
	int n = MASK_SIZE;
	for (int i = 0; i < n*n; i++)
		mask[i] = (float) 1.0f / (n * n);
}


/*
 * main
 */
int main(void) {

  // check params
  if (MASK_RADIUS >= BLOCK_SIZE) {
    printf("ERROR: it holds only if MASK_RADIUS < BLOCK_SIZE!\n");
    return 1;
  }

	int nW = DATA_WIDTH;
  int nH = DATA_HEIGHT;
	int b = BLOCK_SIZE;

	float M[MASK_SIZE*MASK_SIZE]; // const size
	float *A, *B, *A_dev, *B_dev;
	int datasize = nW * nH * sizeof(float);
  int masksize = MASK_SIZE*MASK_SIZE * sizeof(float);

  printf("Data size: %.2f (MB)\n", (float)datasize/(1024.0*1024.0));
	printf("Initializing data...\n");
	A = (float *) malloc(datasize);
	B = (float *) malloc(datasize);

	// initialize data
	for (int i = 0; i < nH; i++)
		for (int j = 0; j < nW; j++)
			A[i*nW+j] = rand()%10;

  // initialize mask 
	Avg_mask(M);

#if DEBUG
	// print data
	printf("Print matrix A...\n");
	for (int i = 0; i < nH; i++) {
    if (i%8 == 0 && i>0)
      printf("\n");

		for (int j = 0; j < nW; j++)
      if (j%8 == 0 && j>0)
			  printf(" %0.0f ", A[i*nW+j]);
      else
        printf("%0.0f ", A[i*nW+j]);
		printf("\n");
	}

	printf("Print matrix M ...\n");
	for (int i = 0; i < MASK_SIZE; i++) {
		for (int j = 0; j < MASK_SIZE; j++)
			  printf(" %1.2f ", M[i * MASK_SIZE + j]);
		printf("\n");
	}
#endif

	// cuda allocation 
	CHECK(cudaMemcpyToSymbol(M_dev, M, masksize));
	CHECK(cudaMalloc((void **) &A_dev, datasize));
	CHECK(cudaMalloc((void **) &B_dev, datasize));
	CHECK(cudaMemcpy(A_dev, A, datasize, cudaMemcpyHostToDevice));
	
	// block, grid dims, kernel
	dim3 block(b, b);
	dim3 grid((nW+b-1)/b, (nH+b-1)/b);
  double iStart, iElaps;
	iStart = seconds();
	conv2D<<<grid, block>>>(A_dev, B_dev);
  cudaDeviceSynchronize();
  iElaps = seconds() - iStart;
	printf("\nconv2D<<<(%d,%d), (%d,%d)>>> elapsed time %f sec \n\n", grid.x, grid.y, block.x, block.y, iElaps);
	CHECK(cudaGetLastError());

	CHECK(cudaMemcpy(B, B_dev, datasize, cudaMemcpyDeviceToHost));

#if DEBUG
	// print out data
	printf("Print results...\n");
	for (int i = 0; i < nH; i++) {
    if (i%8 == 0 && i>0)
      printf("\n");
		for (int j = 0; j < nW; j++)
      if (j%8 == 0 && j>0)
			  printf(" %0.2f ", B[i*nW+j]);
      else
        printf("%0.2f ", B[i*nW+j]);
		printf("\n");
	}
#endif

	cudaFree(A_dev);
	cudaFree(B_dev);
  cudaDeviceReset();
	free(A);
	free(B);
	return 0;
}



In [ ]:
# Compilazione ed esecuzione
!nvcc -arch=sm_70  src/conv2D.cu -o conv2D
!./conv2D